In [1]:
%pip install langchain

  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.42-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain-0.3.25-py3-none

## 组件

### Prompt Templates

Prompt Templates （提示词模板）​用于创建动态提示，可根据用户输入进行定制。它就像是一个 “填空模板”，定义好输入变量和模板内容，运行时将具体值填入变量位置生成完整提示。

In [1]:
from langchain import PromptTemplate
# 定义提示模板，input_variables指定输入变量，template为模板内容
prompt = PromptTemplate(
    input_variables=["name"],
    template="Hello, {name}! Welcome to LangChain."
)
# 使用format方法传入具体值生成提示
output = prompt.format(name="Alice")
print(output) 

Hello, Alice! Welcome to LangChain.


### Models（模型）

LangChain 支持多种模型，可轻松集成到应用中。如使用 OpenAI 的 GPT 模型，代码如下：

```python
from langchain.llms import OpenAI
# 初始化OpenAI模型，需设置API Key
llm = OpenAI(api_key="your_openai_api_key")
# 使用模型生成文本
response = llm("Tell me a joke.")
print(response) 
```

本地部署的ollama模型可以使用


In [8]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(
            # base_url="http://localhost:11434", # windows要加这个选项，mac不加
            model="deepseek-r1:7b")
llm.invoke("你是谁")

'<think>\n\n</think>\n\n您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。'

### Output Parsers（输出解析器）​
Output Parsers 帮助处理模型输出，使其更符合应用使用需求。例如，模型输出可能是原始文本，输出解析器可将其解析为结构化数据（如 JSON 格式）。​

DeepSeek R1会有思考过程，问题简单时打印的结果think和answer高度重复，我们可以自行过滤：

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.output_parsers import RegexParser

llm = OllamaLLM(
            # base_url="http://localhost:11434", # windows要加这个选项，mac不加
            model="deepseek-r1:7b")
raw_output = llm.invoke("你是谁")

# 定义正则表达式解析器，提取冒号后的内容
# 为什么不对？pattern = r"<think>(.*?)</think>\s*(.*)" # 默认不开DOTALL
pattern = r"<think>([\s\S]*?)</think>\s*([\s\S]*)"
output_keys = ["thought", "answer"]

parser = RegexParser(
    regex=pattern,
    output_keys=output_keys,
    default_output_key="answer",
)
parsed = parser.parse(raw_output)

print("Thought:", parsed["thought"])
print("Answer:", parsed["answer"])


Thought: 


Answer: 您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。


## 链

Chain 用于组合多个组件，完成更复杂的任务。例如将提示模板和模型组合成一个链，根据输入生成输出。​

LangChain 使用了「管道操作符」|（pipe operator），让多个组件组合起来更直观。

在内部，其实是通过实现了 `__or__()` 魔术方法（Python 的 | 运算符），将组件自动连接成一个链。


In [16]:
from langchain import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain.chains import LLMChain

# 定义提示模板，input_variables指定输入变量，template为模板内容
prompt = PromptTemplate(
    input_variables=["name"],
    template="你好，我是{name}，请以我的名字呼唤我，并告诉我你是谁"
)

llm = OllamaLLM(
            # base_url="http://localhost:11434", # windows要加这个选项，mac不加
            model="deepseek-r1:7b")
chain = prompt | llm
# chain = LLMChain(llm=llm, prompt=prompt)

chain.invoke({"name":"Alice"})

'<think>\n好，我现在需要处理用户发来的这条消息：“你好，我是Alice，请以我的名字呼唤我，并告诉我你是谁”。首先，我要理解用户的请求是什么。看起来用户希望我用“Alice”作为问候的称呼，并且明确说明自己是Alice。\n\n接下来，我要确认我的角色和职责。作为一个智能助手，我需要保持友好、专业并且准确。所以，在回应时，我会开头以“你好，我是Alice”来称呼对方，这样既符合用户的请求，又显得礼貌。\n\n然后，我需要详细解释一下我是谁，包括我的功能、用途以及未来提供的服务，比如信息查询、学习建议等。这样用户能更好地了解我的作用和提供的帮助范围。\n\n同时，我要确保语言简洁明了，避免使用复杂的术语或过多的说明，让用户能够轻松理解。此外，语气要友好，让用户感到被重视和支持。\n\n最后，我需要总结一下，确认用户的请求已经被满足，并邀请用户提供更多信息或者提出具体的问题，这样可以进一步促进互动。\n\n在实际操作中，可能会遇到用户没有明确表达的需求，比如是否有特定的话题想探讨或是否需要帮助解决某个问题。因此，在回应中要保持开放的态度，以便根据后续的交流调整内容和提供更有针对性的帮助。\n</think>\n\n你好，我是Alice，一个由中国的深度求索（DeepSeek）公司开发的智能助手。我擅长通过思考来帮您解答复杂的数学、代码和逻辑推理等理工类问题，也可以辅助您进行信息查询。请问有什么可以帮到您的？'

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import Runnable
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

# 1. 构造 prompt 和 LLM
prompt = PromptTemplate.from_template("你是一个助手。\n历史记录:\n{chat_history}\n\n用户: {input}\n助手:")
llm = OllamaLLM(
            # base_url="http://localhost:11434", # windows要加这个选项，mac不加
            model="deepseek-r1:7b")

# 2. 组合成一个 LLMChain
chain = prompt | llm

# 3. 用 message history 包装
conversation = RunnableWithMessageHistory(
    chain,
    lambda session_id: InMemoryChatMessageHistory(),  # 每个 session 都有自己的历史
    input_messages_key="input",
    history_messages_key="chat_history"
)

# 4. 使用方式
session_id = "student-session-1"
response = conversation.invoke({"input": "你好"}, config={"configurable": {"session_id": session_id}})
print(response)


<think>
好，用户发来了“你好”，我应该用中文回应他。首先，我需要礼貌地回应他的问候，然后询问他的具体需求是什么。这样可以让对话继续下去，并且帮助我更好地理解用户的需求。

在回应的时候，要保持友好和亲切，避免使用太正式的语言，让对话感觉自然。接着，提出一个开放性的问题，让用户进一步说明他需要什么帮助。比如问：“有什么我可以为您效劳的？”或者类似的表达。

这样既表达了关心，又引导用户明确他们的需求，确保后续的对话能够顺利进行。
</think>

你好！有什么我可以为您效劳的？


RetrievalQA是langchain中专门支持RAG的chain类型，也将弃用。但其语义相对清晰，也予以讲解。
```python
RetrievalQA.from_chain_type( # 使用旧接口
            llm,
            retriever=vectorstore.as_retriever(),
            chain_type="stuff"
        )
```


较新的接口：
```
document_chain = create_stuff_documents_chain(llm, primary_prompt)
qa_chain = create_retrieval_chain(retriever, document_chain)
```



### 文件加载器

In [17]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
import os

SUPPORTED_EXTS = {
    '.txt': lambda p: TextLoader(p, encoding='utf-8'),
    '.pdf': PyPDFLoader
}
file_path = './docs/考试时间.txt'

ext = os.path.splitext(file_path)[1].lower()
if ext not in SUPPORTED_EXTS:
    raise ValueError(f"不支持的文件格式: {ext}")
try:
    loader = SUPPORTED_EXTS[ext](file_path)
    doc = loader.load()
except Exception as e:
    raise RuntimeError(f"加载文件失败: {file_path}") from e


### 向量存储

In [20]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

embeddings = OllamaEmbeddings(
            # base_url="http://localhost:11434", # windows要加这个选项，mac不加
            model='nomic-embed-text')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(doc)
vectorstore = Chroma.from_documents(texts, embeddings)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr",  # or "similarity"
    search_kwargs={
        "k": 5,                 # 返回前k个相似文档
        "score_threshold": 0.7, # 相似度分数阈值（可选，部分vectorstore支持）
        "fetch_k": 20,          # 用于MMR时：从top-N中选择（比如先取前20个再挑5个）
        "lambda_mult": 0.5      # MMR中控制多样性和平衡性的超参数，score = λ * 新文档相似度 - (1 - λ) * 已选文档相似度，越高就找越相关的
    }
)


In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

DEFAULT_PROMPT = """
你是一个智能助手，擅长回答各种问题。
你需要依据以下提供的上下文信息回答用户的问题。
如果上下文信息不足，返回“未找到参考文档”。

上下文信息:
{context}

用户问题:
{input}

回答:
"""

primary_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", DEFAULT_PROMPT),
                    ("human", "{input}"),
                ]
            )

document_chain = create_stuff_documents_chain(llm, primary_prompt)
qa_chain = create_retrieval_chain(retriever, document_chain)
question = "你好，你是谁"
qa_chain.invoke({"input": question})

{'input': '你好，你是谁',
 'context': [Document(metadata={'source': './docs/考试时间.txt'}, page_content='课程考试在第15周或者第16周'),
  Document(metadata={'source': './docs/考试时间.txt'}, page_content='课程考试在第15周或者第16周')],
 'answer': '<think>\n好，我现在需要解决用户的查询。用户首先问：“你好，你是谁。” 这是一个常见的问候问题，通常用于自我介绍。\n\n查看上下文信息，发现没有关于课程考试的详细内容，只有时间安排的信息，并且重复了两次，说明可能只是强调考试的时间在第15或16周。\n\n由于问题主要询问我的身份，而不是课程考试相关的内容，所以我应该专注于回答用户的问题。因此，我会简单地回复“你好！我是智能助手。” 这样既回答了问候，又表明了我的功能。\n</think>\n\n你好！我是智能助手。'}